
Real-Time Machine Learning-Based Automated Trading System for Indian Stock Market
Based on the research paper: "Machine Learning-Based Automated Trading Strategies for the Indian Stock Market"

This implementation includes:
1. Real-time data fetching using yfinance and Alpha Vantage
2. LSTM Neural Network for price prediction
3. Random Forest for classification
4. PPO (Proximal Policy Optimization) for reinforcement learning
5. Real-time trading signals and execution
6. Risk management and portfolio tracking
7. Live dashboard and monitoring


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import requests
import time
import threading
from datetime import datetime, timedelta
import json
import warnings
warnings.filterwarnings('ignore')

# Machine Learning Libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Deep Learning Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Real-time plotting
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# [Other Classes unchanged, including RealTimeDataFetcher, TechnicalIndicators, RealTimeFeatureEngineer, RealTimeLSTMPredictor, RealTimeRandomForest, RealTimePPOAgent, RiskManager]

class RealTimeTradingSystem:
    """Main real-time trading system"""

    def __init__(self, symbols, initial_capital=100000, alpha_vantage_key=None):
        self.symbols = symbols
        self.capital = initial_capital
        self.portfolio = {symbol: 0 for symbol in symbols}
        self.cash = initial_capital

        # Initialize components
        self.data_fetcher = RealTimeDataFetcher(alpha_vantage_key)
        self.feature_engineer = RealTimeFeatureEngineer()
        self.lstm_models = {}
        self.rf_models = {}
        self.ppo_agents = {}
        self.risk_manager = RiskManager()

        # Trading data
        self.historical_data = {}
        self.current_prices = {}
        self.signals = {}
        self.trades = []

        # Performance tracking
        self.performance_history = []
        self.running = False

    def initialize_models(self):
        print("Initializing models with historical data...")

        for symbol in self.symbols:
            print(f"Fetching data for {symbol}...")
            hist_data = self.data_fetcher.get_historical_data(symbol, period="2y")
            if hist_data is None or len(hist_data) < 100:
                print(f"Insufficient data for {symbol}")
                continue

            df_features = self.feature_engineer.add_features(hist_data)
            df_with_labels = self.feature_engineer.create_labels(df_features)
            self.historical_data[symbol] = df_with_labels

            lstm_model = RealTimeLSTMPredictor(input_shape=(60, 1))
            lstm_model.train(df_features)
            self.lstm_models[symbol] = lstm_model

            rf_model = RealTimeRandomForest()
            feature_cols = ['SMA_5', 'SMA_10', 'SMA_20', 'RSI', 'MACD', 'BB_Position', 'Volume_Ratio']
            rf_model.train(df_with_labels, feature_cols)
            self.rf_models[symbol] = rf_model

            ppo_agent = RealTimePPOAgent(state_size=len(feature_cols))
            self.ppo_agents[symbol] = ppo_agent

            print(f"Models initialized for {symbol}")

        print("All models initialized!")

    def get_current_signals(self):
        signals = {}

        for symbol in self.symbols:
            if symbol not in self.historical_data:
                continue

            recent_data = self.data_fetcher.get_intraday_data(symbol)
            if recent_data is None or len(recent_data) < 60:
                continue

            df_features = self.feature_engineer.add_features(recent_data)
            latest_row = df_features.iloc[-1]

            lstm_prediction = None
            if symbol in self.lstm_models:
                lstm_prediction = self.lstm_models[symbol].predict_next_price(df_features['Close'])

            rf_signal, rf_prob = 0, [0.33, 0.33, 0.34]
            if symbol in self.rf_models:
                feature_cols = ['SMA_5', 'SMA_10', 'SMA_20', 'RSI', 'MACD', 'BB_Position', 'Volume_Ratio']
                if all(col in df_features.columns for col in feature_cols):
                    rf_signal, rf_prob = self.rf_models[symbol].predict_signal(latest_row)

            ppo_action = 0
            if symbol in self.ppo_agents:
                state = [latest_row[col] for col in feature_cols if col in df_features.columns]
                if len(state) == len(feature_cols):
                    ppo_action = self.ppo_agents[symbol].get_action(state)

            signals[symbol] = {
                'ppo_action': ppo_action,
                'current_price': latest_row['Close'],
                'timestamp': datetime.now()
            }

        return signals

    def execute_trades(self, signals):
        for symbol, signal_data in signals.items():
            current_price = signal_data['current_price']
            action = signal_data['ppo_action']

            if action == 1:
                self.place_buy_order(symbol, current_price)
            elif action == 2:
                self.place_sell_order(symbol, current_price)

    def place_buy_order(self, symbol, price):
        position_size = self.risk_manager.calculate_position_size(self.cash)
        shares = int(position_size / price)

        if shares > 0 and self.cash >= shares * price:
            self.cash -= shares * price
            self.portfolio[symbol] += shares

            trade = {
                'symbol': symbol,
                'action': 'BUY',
                'shares': shares,
                'price': price,
                'timestamp': datetime.now(),
                'value': shares * price
            }
            self.trades.append(trade)
            print(f"BUY {shares} shares of {symbol} at ₹{price:.2f}")

    def place_sell_order(self, symbol, price):
        shares = self.portfolio[symbol]

        if shares > 0:
            self.cash += shares * price
            self.portfolio[symbol] = 0

            trade = {
                'symbol': symbol,
                'action': 'SELL',
                'shares': shares,
                'price': price,
                'timestamp': datetime.now(),
                'value': shares * price
            }
            self.trades.append(trade)
            print(f"SELL {shares} shares of {symbol} at ₹{price:.2f}")

    def calculate_portfolio_value(self):
        total_value = self.cash
        for symbol, shares in self.portfolio.items():
            if shares > 0:
                current_price = self.current_prices.get(symbol, 0)
                total_value += shares * current_price
        return total_value

    def update_performance(self):
        portfolio_value = self.calculate_portfolio_value()
        performance = {
            'timestamp': datetime.now(),
            'portfolio_value': portfolio_value,
            'cash': self.cash,
            'positions': self.portfolio.copy(),
            'return_pct': (portfolio_value - self.capital) / self.capital * 100
        }
        self.performance_history.append(performance)
        print(f"\n--- Performance Update ---")
        print(f"Portfolio Value: ₹{portfolio_value:,.2f}")
        print(f"Cash: ₹{self.cash:,.2f}")
        print(f"Total Return: {performance['return_pct']:.2f}%")
        print(f"Active Positions: {sum(1 for shares in self.portfolio.values() if shares > 0)}")
        print(f"-------------------------\n")

    def run_trading_loop(self, interval=60):
        print("Starting real-time trading system...")
        self.running = True

        while self.running:
            try:
                signals = self.get_current_signals()
                self.signals = signals

                for symbol, signal_data in signals.items():
                    self.current_prices[symbol] = signal_data['current_price']

                self.execute_trades(signals)
                self.update_performance()

                time.sleep(interval)

            except KeyboardInterrupt:
                print("Stopping trading system...")
                self.running = False
                break
            except Exception as e:
                print(f"Error in trading loop: {e}")
                time.sleep(interval)

    def stop_trading(self):
        self.running = False
        print("Trading system stopped.")

    def get_performance_summary(self):
        if not self.performance_history:
            return "No performance data available"

        latest = self.performance_history[-1]
        initial_value = self.capital
        current_value = latest['portfolio_value']

        return {
            'initial_capital': initial_value,
            'current_value': current_value,
            'total_return': (current_value - initial_value) / initial_value * 100,
            'total_trades': len(self.trades),
            'current_positions': sum(1 for shares in self.portfolio.values() if shares > 0),
            'cash_remaining': self.cash
        }

# Usage example
def main():
    symbols = ['RELIANCE', 'TCS', 'HDFCBANK', 'INFY', 'HINDUNILVR']
    trading_system = RealTimeTradingSystem(
        symbols=symbols,
        initial_capital=100000,
        alpha_vantage_key='CHVWFO2DTKHXKUMN'
    )
    trading_system.initialize_models()
    try:
        trading_system.run_trading_loop(interval=300)
    except KeyboardInterrupt:
        trading_system.stop_trading()
    summary = trading_system.get_performance_summary()
    print("\n=== Final Performance Summary ===")
    for key, value in summary.items():
        print(f"{key}: {value}")

if __name__ == "__main__":
    print("Real-Time ML Trading System for Indian Stock Market")
    print("Based on research: 'Machine Learning-Based Automated Trading Strategies'")
    main()


Real-Time ML Trading System for Indian Stock Market
Based on research: 'Machine Learning-Based Automated Trading Strategies'
Initializing models with historical data...
Fetching data for RELIANCE...
Models initialized for RELIANCE
Fetching data for TCS...
Models initialized for TCS
Fetching data for HDFCBANK...


In [ ]:
"""
ML Trading Strategy Backtester
Trains models on pre-2024 data and backtests on 2024 data
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import warnings
from datetime import datetime, timedelta
import time
warnings.filterwarnings('ignore')

# Machine Learning Libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Deep Learning Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Suppress TensorFlow warnings
tf.get_logger().setLevel('ERROR')

class TechnicalIndicators:
    """Technical indicators for feature engineering"""

    @staticmethod
    def calculate_sma(data, window):
        """Simple Moving Average"""
        return data.rolling(window=window).mean()

    @staticmethod
    def calculate_ema(data, window):
        """Exponential Moving Average"""
        return data.ewm(span=window).mean()

    @staticmethod
    def calculate_rsi(data, window=14):
        """Relative Strength Index"""
        delta = data.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    @staticmethod
    def calculate_macd(data, fast=12, slow=26, signal=9):
        """MACD Indicator"""
        ema_fast = data.ewm(span=fast).mean()
        ema_slow = data.ewm(span=slow).mean()
        macd_line = ema_fast - ema_slow
        signal_line = macd_line.ewm(span=signal).mean()
        histogram = macd_line - signal_line
        return macd_line, signal_line, histogram

    @staticmethod
    def calculate_bollinger_bands(data, window=20, num_std=2):
        """Bollinger Bands"""
        sma = data.rolling(window=window).mean()
        std = data.rolling(window=window).std()
        upper_band = sma + (std * num_std)
        lower_band = sma - (std * num_std)
        return upper_band, sma, lower_band

    @staticmethod
    def calculate_volatility(data, window=20):
        """Calculate rolling volatility"""
        return data.pct_change().rolling(window=window).std() * np.sqrt(252)

class DataPreprocessor:
    """Data preprocessing and feature engineering"""

    def __init__(self):
        self.scaler = StandardScaler()
        self.price_scaler = MinMaxScaler()

    def fetch_data(self, symbol, start_date, end_date):
        """Fetch historical data"""
        try:
            ticker = yf.Ticker(f"{symbol}.NS")
            data = ticker.history(start=start_date, end=end_date)
            if len(data) < 100:
                print(f"Warning: Limited data for {symbol}")
            return data
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            return None

    def add_features(self, df):
        """Add technical indicators as features"""
        df = df.copy()

        # Price-based features
        df['Returns'] = df['Close'].pct_change()
        df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))

        # Moving averages
        df['SMA_5'] = TechnicalIndicators.calculate_sma(df['Close'], 5)
        df['SMA_10'] = TechnicalIndicators.calculate_sma(df['Close'], 10)
        df['SMA_20'] = TechnicalIndicators.calculate_sma(df['Close'], 20)
        df['SMA_50'] = TechnicalIndicators.calculate_sma(df['Close'], 50)
        df['EMA_12'] = TechnicalIndicators.calculate_ema(df['Close'], 12)
        df['EMA_26'] = TechnicalIndicators.calculate_ema(df['Close'], 26)

        # Technical indicators
        df['RSI'] = TechnicalIndicators.calculate_rsi(df['Close'])
        macd, signal, histogram = TechnicalIndicators.calculate_macd(df['Close'])
        df['MACD'] = macd
        df['MACD_Signal'] = signal
        df['MACD_Histogram'] = histogram

        # Bollinger Bands
        upper_bb, middle_bb, lower_bb = TechnicalIndicators.calculate_bollinger_bands(df['Close'])
        df['BB_Upper'] = upper_bb
        df['BB_Middle'] = middle_bb
        df['BB_Lower'] = lower_bb
        df['BB_Width'] = (upper_bb - lower_bb) / middle_bb
        df['BB_Position'] = (df['Close'] - lower_bb) / (upper_bb - lower_bb)

        # Volume indicators
        df['Volume_SMA'] = TechnicalIndicators.calculate_sma(df['Volume'], 20)
        df['Volume_Ratio'] = df['Volume'] / df['Volume_SMA']

        # Volatility
        df['Volatility'] = TechnicalIndicators.calculate_volatility(df['Close'])

        # Price position indicators
        df['Price_SMA20_Ratio'] = df['Close'] / df['SMA_20']
        df['Price_SMA50_Ratio'] = df['Close'] / df['SMA_50']
        df['High_Low_Ratio'] = df['High'] / df['Low']

        # Momentum indicators
        df['Price_Change_5'] = df['Close'].pct_change(5)
        df['Price_Change_10'] = df['Close'].pct_change(10)

        return df

    def create_labels(self, df, threshold=0.02):
        """Create trading labels based on future returns"""
        df = df.copy()
        df['Future_Returns'] = df['Close'].shift(-1) / df['Close'] - 1

        # Classification labels: 0=Hold, 1=Buy, 2=Sell
        df['Signal'] = 0
        df.loc[df['Future_Returns'] > threshold, 'Signal'] = 1  # Buy
        df.loc[df['Future_Returns'] < -threshold, 'Signal'] = 2  # Sell

        return df

class LSTMModel:
    """LSTM model for price prediction"""

    def __init__(self, sequence_length=60, units=50, dropout_rate=0.2):
        self.sequence_length = sequence_length
        self.units = units
        self.dropout_rate = dropout_rate
        self.model = None
        self.scaler = MinMaxScaler()
        self.is_trained = False

    def prepare_data(self, df):
        """Prepare data for LSTM"""
        data = df['Close'].values.reshape(-1, 1)
        scaled_data = self.scaler.fit_transform(data)

        X, y = [], []
        for i in range(self.sequence_length, len(scaled_data)):
            X.append(scaled_data[i-self.sequence_length:i, 0])
            y.append(scaled_data[i, 0])

        return np.array(X), np.array(y)

    def build_model(self, input_shape):
        """Build LSTM model"""
        self.model = Sequential([
            LSTM(self.units, return_sequences=True, input_shape=input_shape),
            Dropout(self.dropout_rate),
            LSTM(self.units, return_sequences=True),
            Dropout(self.dropout_rate),
            LSTM(self.units),
            Dropout(self.dropout_rate),
            Dense(25, activation='relu'),
            Dense(1)
        ])

        self.model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )

    def train(self, df, epochs=50, batch_size=32, validation_split=0.2):
        """Train the LSTM model"""
        X, y = self.prepare_data(df)

        if len(X) < 100:
            print("Insufficient data for LSTM training")
            return None

        X = X.reshape((X.shape[0], X.shape[1], 1))

        if self.model is None:
            self.build_model((X.shape[1], 1))

        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )

        history = self.model.fit(
            X, y,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=validation_split,
            callbacks=[early_stopping],
            verbose=0
        )

        self.is_trained = True
        return history

    def predict(self, df):
        """Make predictions"""
        if not self.is_trained:
            return None

        X, _ = self.prepare_data(df)
        X = X.reshape((X.shape[0], X.shape[1], 1))

        scaled_predictions = self.model.predict(X, verbose=0)
        predictions = self.scaler.inverse_transform(scaled_predictions)

        return predictions.flatten()

class RandomForestModel:
    """Random Forest model for signal classification"""

    def __init__(self, n_estimators=100, max_depth=10, random_state=42):
        self.model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=random_state
        )
        self.scaler = StandardScaler()
        self.feature_columns = None
        self.is_trained = False

    def train(self, df, feature_columns, target='Signal'):
        """Train Random Forest model"""
        df_clean = df.dropna()

        if len(df_clean) < 100:
            print("Insufficient data for Random Forest training")
            return False

        X = df_clean[feature_columns]
        y = df_clean[target]

        # Handle any remaining NaN values
        X = X.fillna(X.mean())

        X_scaled = self.scaler.fit_transform(X)

        self.model.fit(X_scaled, y)
        self.feature_columns = feature_columns
        self.is_trained = True

        # Print feature importance
        importance = pd.DataFrame({
            'feature': feature_columns,
            'importance': self.model.feature_importances_
        }).sort_values('importance', ascending=False)

        print(f"Top 5 most important features:")
        print(importance.head())

        return True

    def predict(self, df):
        """Make predictions"""
        if not self.is_trained:
            return np.zeros(len(df)), np.zeros((len(df), 3))

        X = df[self.feature_columns].fillna(df[self.feature_columns].mean())
        X_scaled = self.scaler.transform(X)

        predictions = self.model.predict(X_scaled)
        probabilities = self.model.predict_proba(X_scaled)

        return predictions, probabilities

class SimplePPOAgent:
    """Simplified PPO agent for trading"""

    def __init__(self, state_size, action_size=3, learning_rate=0.0003):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.policy_model = self._build_policy_model()
        self.value_model = self._build_value_model()
        self.is_trained = False

    def _build_policy_model(self):
        """Build policy network"""
        model = Sequential([
            Dense(64, activation='relu', input_shape=(self.state_size,)),
            Dense(32, activation='relu'),
            Dense(self.action_size, activation='softmax')
        ])
        model.compile(optimizer=Adam(learning_rate=self.learning_rate), loss='categorical_crossentropy')
        return model

    def _build_value_model(self):
        """Build value network"""
        model = Sequential([
            Dense(64, activation='relu', input_shape=(self.state_size,)),
            Dense(32, activation='relu'),
            Dense(1, activation='linear')
        ])
        model.compile(optimizer=Adam(learning_rate=self.learning_rate), loss='mse')
        return model

    def get_action(self, state):
        """Get action based on current state"""
        if not self.is_trained:
            return 0  # Hold action

        state = np.reshape(state, [1, self.state_size])
        action_probs = self.policy_model.predict(state, verbose=0)[0]
        return np.random.choice(self.action_size, p=action_probs)

    def get_action_probabilities(self, states):
        """Get action probabilities for multiple states"""
        if not self.is_trained:
            return np.full((len(states), self.action_size), 1/self.action_size)

        return self.policy_model.predict(states, verbose=0)

    def simple_train(self, states, actions, rewards, epochs=10):
        """Simplified training"""
        if len(states) < 50:
            return

        states = np.array(states)
        actions = np.array(actions)
        rewards = np.array(rewards)

        # Normalize rewards
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-8)

        # Convert actions to one-hot encoding
        actions_onehot = tf.keras.utils.to_categorical(actions, self.action_size)

        for _ in range(epochs):
            self.policy_model.fit(states, actions_onehot, epochs=1, verbose=0)
            self.value_model.fit(states, rewards, epochs=1, verbose=0)

        self.is_trained = True

class BacktestEngine:
    """Backtesting engine"""

    def __init__(self, initial_capital=100000):
        self.initial_capital = initial_capital
        self.capital = initial_capital
        self.positions = {}
        self.trades = []
        self.portfolio_history = []
        self.transaction_cost = 0.001  # 0.1% transaction cost

    def reset(self):
        """Reset the backtester"""
        self.capital = self.initial_capital
        self.positions = {}
        self.trades = []
        self.portfolio_history = []

    def place_order(self, symbol, action, price, quantity, timestamp):
        """Place an order"""
        if action == 'BUY':
            cost = quantity * price * (1 + self.transaction_cost)
            if self.capital >= cost:
                self.capital -= cost
                self.positions[symbol] = self.positions.get(symbol, 0) + quantity

                trade = {
                    'symbol': symbol,
                    'action': action,
                    'quantity': quantity,
                    'price': price,
                    'timestamp': timestamp,
                    'cost': cost
                }
                self.trades.append(trade)
                return True

        elif action == 'SELL':
            if self.positions.get(symbol, 0) >= quantity:
                revenue = quantity * price * (1 - self.transaction_cost)
                self.capital += revenue
                self.positions[symbol] -= quantity

                trade = {
                    'symbol': symbol,
                    'action': action,
                    'quantity': quantity,
                    'price': price,
                    'timestamp': timestamp,
                    'revenue': revenue
                }
                self.trades.append(trade)
                return True

        return False

    def calculate_portfolio_value(self, current_prices):
        """Calculate current portfolio value"""
        total_value = self.capital
        for symbol, quantity in self.positions.items():
            if quantity > 0:
                total_value += quantity * current_prices.get(symbol, 0)
        return total_value

    def record_portfolio_state(self, timestamp, current_prices):
        """Record current portfolio state"""
        portfolio_value = self.calculate_portfolio_value(current_prices)

        state = {
            'timestamp': timestamp,
            'portfolio_value': portfolio_value,
            'capital': self.capital,
            'positions': self.positions.copy(),
            'return_pct': (portfolio_value - self.initial_capital) / self.initial_capital * 100
        }

        self.portfolio_history.append(state)

class TradingStrategy:
    """Main trading strategy class"""

    def __init__(self, symbols, initial_capital=100000):
        self.symbols = symbols
        self.initial_capital = initial_capital

        # Initialize components
        self.data_processor = DataPreprocessor()
        self.lstm_models = {}
        self.rf_models = {}
        self.ppo_agents = {}
        self.backtest_engine = BacktestEngine(initial_capital)

        # Data storage
        self.train_data = {}
        self.test_data = {}

        # Feature columns
        self.feature_columns = [
            'SMA_5', 'SMA_10', 'SMA_20', 'RSI', 'MACD', 'MACD_Signal',
            'BB_Position', 'BB_Width', 'Volume_Ratio', 'Volatility',
            'Price_SMA20_Ratio', 'Price_Change_5', 'Price_Change_10'
        ]

    def load_data(self, train_start='2020-01-01', train_end='2023-12-31',
                  test_start='2024-01-01', test_end='2024-12-31'):
        """Load training and testing data"""
        print("Loading data...")

        for symbol in self.symbols:
            print(f"Fetching data for {symbol}...")

            # Fetch training data
            train_data = self.data_processor.fetch_data(symbol, train_start, train_end)
            if train_data is not None:
                train_data = self.data_processor.add_features(train_data)
                train_data = self.data_processor.create_labels(train_data)
                self.train_data[symbol] = train_data

            # Fetch testing data
            test_data = self.data_processor.fetch_data(symbol, test_start, test_end)
            if test_data is not None:
                test_data = self.data_processor.add_features(test_data)
                test_data = self.data_processor.create_labels(test_data)
                self.test_data[symbol] = test_data

        print(f"Data loaded for {len(self.train_data)} symbols")

    def train_models(self):
        """Train all models"""
        print("Training models...")

        for symbol in self.symbols:
            if symbol not in self.train_data:
                continue

            print(f"Training models for {symbol}...")
            train_df = self.train_data[symbol].dropna()

            if len(train_df) < 200:
                print(f"Insufficient data for {symbol}")
                continue

            # Train LSTM
            lstm_model = LSTMModel(sequence_length=60)
            lstm_model.train(train_df, epochs=30, batch_size=32)
            self.lstm_models[symbol] = lstm_model

            # Train Random Forest
            rf_model = RandomForestModel(n_estimators=100, max_depth=15)
            rf_model.train(train_df, self.feature_columns)
            self.rf_models[symbol] = rf_model

            # Train PPO (simplified)
            ppo_agent = SimplePPOAgent(state_size=len(self.feature_columns))

            # Create training data for PPO
            states = []
            actions = []
            rewards = []

            for i in range(len(train_df) - 10):
                if i < len(self.feature_columns):
                    continue

                state = train_df.iloc[i][self.feature_columns].values
                if not np.any(np.isnan(state)):
                    states.append(state)
                    actions.append(train_df.iloc[i]['Signal'])
                    rewards.append(train_df.iloc[i]['Returns'])

            if len(states) > 100:
                ppo_agent.simple_train(states, actions, rewards)

            self.ppo_agents[symbol] = ppo_agent

            print(f"Models trained for {symbol}")

    def generate_signals(self, symbol, current_data):
          """Generate trading signals using only PPO"""
          if symbol not in self.ppo_agents:
              return 0, {'reason': 'No PPO agent for symbol'}

          current_row = current_data.iloc[-1]
          ppo_signal = 0

          try:
              state = current_row[self.feature_columns].values
              if not np.any(np.isnan(state)):
                  ppo_signal = self.ppo_agents[symbol].get_action(state)
          except:
              return 0, {'reason': 'Invalid state for PPO'}

          signal_details = {
              'ppo_signal': ppo_signal,
              'reason': 'Only PPO decision used'
          }

          return ppo_signal, signal_details


    def run_backtest(self, position_size_pct=0.1, min_hold_days=1):
        """Run the backtest"""
        print("Running backtest...")

        self.backtest_engine.reset()

        # Get all test dates
        all_dates = set()
        for symbol in self.symbols:
            if symbol in self.test_data:
                all_dates.update(self.test_data[symbol].index.date)

        all_dates = sorted(all_dates)

        # Track position ages
        position_ages = {}

        for date in all_dates:
            current_prices = {}

            # Process each symbol
            for symbol in self.symbols:
                if symbol not in self.test_data:
                    continue

                test_df = self.test_data[symbol]

                # Get data up to current date
                current_data = test_df[test_df.index.date <= date]

                if len(current_data) < 70:  # Need enough data for LSTM
                    continue

                current_price = current_data.iloc[-1]['Close']
                current_prices[symbol] = current_price

                # Generate signal
                signal, signal_details = self.generate_signals(symbol, current_data)

                # Check position age
                if symbol not in position_ages:
                    position_ages[symbol] = 0
                else:
                    position_ages[symbol] += 1

                # Execute trades based on signal
                current_position = self.backtest_engine.positions.get(symbol, 0)
                position_size = int(self.backtest_engine.capital * position_size_pct / current_price)

                if signal == 1 and current_position == 0:  # Buy signal
                    if self.backtest_engine.place_order(symbol, 'BUY', current_price, position_size, date):
                        position_ages[symbol] = 0

                elif signal == 2 and current_position > 0:  # Sell signal
                    if position_ages[symbol] >= min_hold_days:
                        self.backtest_engine.place_order(symbol, 'SELL', current_price, current_position, date)
                        position_ages[symbol] = 0

            # Record portfolio state
            self.backtest_engine.record_portfolio_state(date, current_prices)

        print("Backtest completed!")

    def analyze_results(self):
        """Analyze backtest results"""
        if not self.backtest_engine.portfolio_history:
            print("No backtest results to analyze")
            return

        # Create performance DataFrame
        performance_df = pd.DataFrame(self.backtest_engine.portfolio_history)
        performance_df['timestamp'] = pd.to_datetime(performance_df['timestamp'])
        performance_df.set_index('timestamp', inplace=True)

        # Calculate metrics
        final_value = performance_df['portfolio_value'].iloc[-1]
        total_return = (final_value - self.initial_capital) / self.initial_capital * 100

        # Calculate daily returns
        daily_returns = performance_df['portfolio_value'].pct_change().dropna()

        # Risk metrics
        volatility = daily_returns.std() * np.sqrt(252) * 100
        sharpe_ratio = (daily_returns.mean() * 252) / (daily_returns.std() * np.sqrt(252)) if daily_returns.std() > 0 else 0

        # Drawdown calculation
        rolling_max = performance_df['portfolio_value'].expanding().max()
        drawdown = (performance_df['portfolio_value'] - rolling_max) / rolling_max * 100
        max_drawdown = drawdown.min()

        # Win rate
        winning_trades = sum(1 for trade in self.backtest_engine.trades
                           if trade['action'] == 'SELL' and
                           any(t['action'] == 'BUY' and t['symbol'] == trade['symbol']
                               for t in self.backtest_engine.trades))

        total_trades = len([t for t in self.backtest_engine.trades if t['action'] == 'SELL'])
        win_rate = (winning_trades / total_trades * 100) if total_trades > 0 else 0

        # Print results
        print("\n" + "="*50)
        print("BACKTEST RESULTS")
        print("="*50)
        print(f"Initial Capital: ₹{self.initial_capital:,.2f}")
        print(f"Final Portfolio Value: ₹{final_value:,.2f}")
        print(f"Total Return: {total_return:.2f}%")
        print(f"Annualized Volatility: {volatility:.2f}%")
        print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
        print(f"Maximum Drawdown: {max_drawdown:.2f}%")
        print(f"Total Trades: {len(self.backtest_engine.trades)}")
        print(f"Win Rate: {win_rate:.2f}%")
        print("="*50)

        # Plot results
        self.plot_results(performance_df, drawdown)

        return {
            'total_return': total_return,
            'volatility': volatility,
            'sharpe_ratio': sharpe_ratio,
            'max_drawdown': max_drawdown,
            'total_trades': len(self.backtest_engine.trades),
            'win_rate': win_rate,
            'final_value': final_value
        }

    def plot_results(self, performance_df, drawdown):
        """Plot backtest results"""
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))

        # Portfolio value over time
        axes[0, 0].plot(performance_df.index, performance_df['portfolio_value'])
        axes[0, 0].set_title('Portfolio Value Over Time')
        axes[0, 0].set_ylabel('Portfolio Value (₹)')
        axes[0, 0].grid(True)

        # Returns over time
        axes[0, 1].plot(performance_df.index, performance_df['return_pct'])
        axes[0, 1].set_title('Cumulative Returns')
        axes[0, 1].set_ylabel('Returns (%)')
        axes[0, 1].grid(True)

        # Drawdown
        axes[1, 0].fill_between(performance_df.index, drawdown, 0, alpha=0.3, color='red')
        axes[1, 0].set_title('Drawdown')
        axes[1, 0].set_ylabel('Drawdown (%)')
        axes[1, 0].grid(True)

        # Trade distribution
        trade_actions = [trade['action'] for trade in self.backtest_engine.trades]
        trade_counts = pd.Series(trade_actions).value_counts()
        axes[1, 1].bar(trade_counts.index, trade_counts.values)
        axes[1, 1].set_title('Trade Distribution')
        axes[1, 1].set_ylabel('Number of Trades')

        plt.tight_layout()
        plt.show()

def main():
    """Main function to run the backtest"""

    symbols =[ 'RELIANCE','ICICIBANK', 'INFY', 'HDFCBANK']

    print("Initializing Trading Strategy Backtest...")
    strategy = TradingStrategy(symbols, initial_capital=100000)

    # Step 1: Load data
    strategy.load_data(
        train_start='2020-01-01', train_end='2023-12-31',
        test_start='2024-01-01', test_end='2024-12-31'
    )

    # Step 2: Train models
    strategy.train_models()

    # Step 3: Run backtest
    strategy.run_backtest(position_size_pct=0.1, min_hold_days=1)

    # Step 4: Analyze results
    strategy.analyze_results()
if __name__ == '__main__':
    main()
